In [1]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3.2")
response = llm.invoke("hi, ok")
print(response)

How can I assist you today?


In [3]:
from langchain_ollama import OllamaEmbeddings

# Инициализация эмбеддинговой модели
embeddings = OllamaEmbeddings(model="all-minilm:33m")

# Список текстов для преобразования в векторы
texts = [
    "Что такое LangChain?",
    "Как использовать Ollama для создания эмбеддингов?",
    "Какие модели поддерживает Ollama?"
]

# Получение векторов для списка текстов
vectors = embeddings.embed_documents(texts)

# Вывод векторов
for i, vector in enumerate(vectors):
    print(f"Вектор для текста {i+1}: {vector}")

Вектор для текста 1: [0.018184064, 0.015606684, -0.012226378, -0.0132663995, -0.08529381, -0.038127303, 0.08370108, 0.0069631166, 0.0624118, 0.005192256, 0.0032863321, 0.012073303, -0.018539649, -0.057587743, 0.032785907, 0.03189397, -0.043051288, 0.034962915, -0.03587919, -0.101649635, -0.09072711, 0.01669838, 0.05374657, 0.037583906, -0.022780284, -0.118925184, 0.023636894, -0.010320635, -0.040740084, -0.0140410075, -0.07661302, 0.019521216, -0.09955438, 0.046540223, 0.028736936, 0.004788611, -0.073173754, -0.058698054, -0.0064741047, -0.018147144, 0.0028852373, -0.036850773, 0.0146198645, -0.06371735, 0.024002606, -0.05622523, -0.060398664, 0.0060183625, -0.0646925, 0.071857624, -0.11664924, -0.00061142264, 0.001888581, 0.05302631, 0.101434916, -0.07745208, 0.05860222, -0.017180381, -0.017619874, -0.06107898, 0.052794617, 0.024668382, -0.043501146, 0.05000258, 0.03136295, 0.056956533, -0.03202284, 0.051034987, 0.0010633061, 0.03185863, -0.03545366, -0.013606721, 0.00520748, 0.052190

In [5]:
import numpy as np
vectors = np.array(vectors)

In [9]:
np.linalg.norm(vectors, axis=1)

array([1.00000002, 1.00000011, 0.99999985])

In [ ]:
new_matr = vectors/np.linalg.norm(vectors, axis=1, keepdims=True)

array([[ 0.01818406,  0.01560668, -0.01222638, ..., -0.05385087,
         0.03002671, -0.1237399 ],
       [-0.05530169, -0.01018599, -0.09763478, ...,  0.00967609,
         0.06542225, -0.08340779],
       [-0.01067773, -0.02184191, -0.03008259, ...,  0.02676324,
        -0.01109903, -0.08038011]])